<a href="https://colab.research.google.com/github/soyoung96/algorithm/blob/master/ResNet50_CIFAR10_Basic_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### <b>ResNet50 모델 아키텍처 정의</b>


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os


# ResNet50을 위해 최대한 간단히 수정한 BasicBlock 클래스
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 1x1 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 1x1 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv3 = nn.Conv2d(planes, planes*4, kernel_size=1,  bias=False)
        self.bn3 = nn.BatchNorm2d(planes*4) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1 or in_planes != 4*planes: # stride가 1이 아니라면, identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, 4 * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(4*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Conv2d(4*512, num_classes,kernel_size=1) #바꿈

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes*4 # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out) # 출력: [batch_size, 512, 4, 4]
        out = F.avg_pool2d(out, 4) # 출력: [batch_size, 512, 1, 1]
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet50():
    return ResNet(BasicBlock, [3, 4, 6, 3])

#### <b>데이터셋(Dataset) 다운로드 및 불러오기</b>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.pylab as plt2
import os



#Check GPU, connect to it if it is available 
device = ''
if torch.cuda.is_available():
	device = 'cuda'
	print("CUDA is available. GPU will be used for training.")
else:
	device = 'cpu'


BEST_ACCURACY = 0

# Preparing Data
print("==> Prepairing data ...")
#Transformation on train data
transform_train = transforms.Compose([
	transforms.RandomCrop(32, padding=4),
	transforms.RandomHorizontalFlip(),
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
	])

#transformation on validation data
transform_validation = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
	])

#Download Train and Validation data and apply transformation
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
validation_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_validation)

#Put data into trainloader, specify batch_size
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=100, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#Function to show CIFAR images
def show_data(image):
	plt.imshow(np.transpose(image[0], (1, 2, 0)), interpolation='bicubic')
	plt.show()

#show_data(train_data[0])


#Need to import model a model
model = ResNet50()
#model = ResNet34()
#model = CNN_batch()
#Pass model to GPU
model = model.to(device)
model.train()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

length_train = len(train_data)
length_validation = len(validation_data)
#print(length_train)
#print(len(train_loader))
num_classes = 10

#Training
def train(epochs):
	global BEST_ACCURACY
	dict = {'Train Loss':[], 'Train Acc':[], 'Validation Loss':[], 'Validation Acc':[]}
	for epoch in range(epochs):
		print("\nEpoch:", epoch+1, "/", epochs)
		cost = 0
		correct = 0
		total = 0
		woha = 0

		for i, (x,y) in enumerate(train_loader):
			woha += 1
			model.train()
			x, y = x.to(device), y.to(device)
			optimizer.zero_grad()
			yhat = model(x)
			yhat = yhat.reshape(-1, 10)
			loss = criterion(yhat, y)
			loss.backward()
			optimizer.step()
			cost += loss.item()

			_, yhat2 = torch.max(yhat.data, 1)
			correct += (yhat2 == y).sum().item()
			total += y.size(0)

		my_loss = cost/len(train_loader)
		my_accuracy = 100*correct/length_train

		dict['Train Loss'].append(my_loss)
		dict['Train Acc'].append(my_accuracy)

		print('Tain Loss:', my_loss)
		print('Train Accuracy:', my_accuracy,'%')


		cost = 0
		correct = 0

		with torch.no_grad():
			for x, y in validation_loader:
				x, y = x.to(device), y.to(device)
				model.eval()
				yhat = model(x)
				yhat = yhat.reshape(-1, 10)
				loss = criterion(yhat, y)
				cost += loss.item()
				
				_, yhat2 = torch.max(yhat.data, 1)
				correct += (yhat2 == y).sum().item()

		my_loss = cost/len(validation_loader)
		my_accuracy = 100*correct/length_validation

		dict['Validation Loss'].append(my_loss)
		dict['Validation Acc'].append(my_accuracy)

		print('Validation Loss:', my_loss)
		print('Validation Accuracy:', my_accuracy,'%')

		#Save the model if you get best accuracy on validation data
		if my_accuracy > BEST_ACCURACY:
			BEST_ACCURACY = my_accuracy
			print('Saving the model ...')
			model.eval()
			if not os.path.isdir('checkpoint'):
			    os.mkdir('checkpoint')
			torch.save(model.state_dict(), './checkpoint/resnet50.pth')

	print("TRAINING IS FINISHED !!!")
	return dict

#Start training
results = train(40)



CUDA is available. GPU will be used for training.
==> Prepairing data ...
Files already downloaded and verified
Files already downloaded and verified

Epoch: 1 / 40
Tain Loss: 1.811822697329704
Train Accuracy: 34.978 %
Validation Loss: 1.3918440628051758
Validation Accuracy: 50.17 %
Saving the model ...

Epoch: 2 / 40
Tain Loss: 1.2346497118625495
Train Accuracy: 55.392 %
Validation Loss: 1.0545808124542235
Validation Accuracy: 62.5 %
Saving the model ...

Epoch: 3 / 40
Tain Loss: 0.9595536114004872
Train Accuracy: 66.112 %
Validation Loss: 1.0120917254686355
Validation Accuracy: 66.87 %
Saving the model ...

Epoch: 4 / 40
Tain Loss: 0.7841031276966299
Train Accuracy: 72.566 %
Validation Loss: 0.7687614697217942
Validation Accuracy: 73.18 %
Saving the model ...

Epoch: 5 / 40
Tain Loss: 0.6524009110830019
Train Accuracy: 77.314 %
Validation Loss: 0.8402917361259461
Validation Accuracy: 73.25 %
Saving the model ...

Epoch: 6 / 40
Tain Loss: 0.5565776464426914
Train Accuracy: 80.698 %
Va

#### <b>환경 설정 및 학습(Training) 함수 정의</b>

In [ ]:
plt.figure(1)
plt.plot(results['Train Loss'], 'b', label = 'training loss')
plt.plot(results['Validation Loss'], 'r', label = 'validation loss')
plt.title("LOSS")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(['training set', 'validation set'], loc='center right')
plt.savefig('Loss_ResNet50.png', dpi=300, bbox_inches='tight')

plt.figure(2)
plt.plot(results['Train Acc'], 'b', label = 'training accuracy')
plt.plot(results['Validation Acc'], 'r', label = 'validation accuracy')
plt.title("ACCURACY")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(['training set', 'validation set'], loc='center right')
plt.savefig('Accuracy_ResNet50.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

NameError: ignored

#### <b>학습(Training) 진행</b>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.pylab as plt2
import os

#Check GPU, connect to it if it is available 
device = ''
if torch.cuda.is_available():
	device = 'cuda'
	print("CUDA is available. GPU will be used for testing.")
else:
	device = 'cpu'


BEST_ACCURACY = 0

# Preparing Data
print("==> Prepairing data ...")
#transformation on validation data
transform_validation = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
	])

#Download Validation data and apply transformation
validation_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_validation)

#Put data into loader, specify batch_size
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=100, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#Function to show CIFAR images
def show_data(image):
	plt.imshow(np.transpose(image[0], (1, 2, 0)), interpolation='bicubic')
	plt.show()


model = ResNet50()
#model = ResNet34()
model = model.to(device)
print("Upload the model ...")
assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
model.load_state_dict(torch.load('./checkpoint/resnet50.pth'))
model.eval()


def accuracy(output, target, topk=(1,)):
	"""Computes the precision@k for the specified values of k"""
	maxk = max(topk)
	
	_, pred = output.topk(maxk, 1, True, True)
	pred = pred.t()
	correct = pred.eq(target.view(1, -1).expand_as(pred))
	
	res = []
	for k in topk:
	    correct_k = correct[:k].view(-1).float().sum(0)
	    res.append(correct_k)
	return res

def test():
	model.eval()
	with torch.no_grad():
		accuracy1 = 0
		accuracy5 = 0
		for x,y in validation_loader:
			x, y = x.to(device), y.to(device)
			model.eval()
			yhat = model(x)
			yhat = yhat.reshape(-1, 10)
			a1, a5 = accuracy(yhat, y, topk=(1,5))
			accuracy1 += a1 
			accuracy5 += a5

		return (accuracy1/len(validation_data)).item(), (accuracy5/(len(validation_data))).item()


acc1, acc5 = test()

print("--------------------------")
print("|       ResNet50         |")
print("--------------------------")
print("| TOP1 Accuracy:", format(100*acc1, '.4f'), "|")
print("| TOP5 Accuracy:", format(100*acc5, '.4f'), "|")
print("--------------------------\n")


[ Train epoch: 0 ]


RuntimeError: ignored